In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

import h2o

from h2o.automl import H2OAutoML

In [6]:
train_df = pd.read_csv('../data/processed/learn.csv')
train_df.drop(columns=['process_end_time', 'final_mes_time'], inplace=True)
train_df.head()

,OV,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83
0,111.2886,22.98,1.37,797.20,0.67,0.15,0.08,67.15,9.86,37.23,...,0.1,0.04,6.4,0.58,0.0,4.1,0.0,0.01,0.16,0.04
1,201.5617,22.45,9.40,803.80,0.68,0.15,0.08,56.11,5.27,33.98,...,0.1,0.04,6.4,0.58,0.0,4.1,0.0,0.01,0.16,0.04
2,111.9410,22.03,1.14,594.52,0.64,0.14,0.08,51.57,6.15,10.44,...,0.1,0.04,6.4,0.58,0.0,4.1,0.0,0.01,0.16,0.04
3,182.3914,22.98,1.37,797.20,0.67,0.15,0.08,67.15,9.86,37.23,...,0.1,0.04,6.4,0.58,0.0,4.1,0.0,0.01,0.16,0.04
4,196.2703,22.98,1.37,797.20,0.67,0.15,0.08,67.15,9.86,37.23,...,0.1,0.04,6.4,0.58,0.0,4.1,0.0,0.01,0.16,0.04


In [7]:
test_df = pd.read_csv('../data/processed/test.csv')
test_df.drop(columns=['process_end_time', 'final_mes_time'], inplace=True)
test_df.head()


,OV,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83
0,130.1202,23.62,9.35,705.91,0.64,0.14,0.08,127.48,4.25,7.30,...,0.03,0.26,6.5,0.2,0.0,2.7,0.02,0.01,0.25,0.04
1,115.6764,23.62,9.35,705.91,0.64,0.14,0.08,127.48,4.25,7.30,...,0.03,0.26,6.5,0.2,0.0,2.7,0.02,0.01,0.25,0.04
2,104.8819,23.62,9.35,705.91,0.64,0.14,0.08,127.48,4.25,7.30,...,0.03,0.26,6.5,0.2,0.0,2.7,0.02,0.01,0.25,0.04
3,108.7957,24.35,4.92,635.52,0.64,0.14,0.09,78.09,3.22,6.29,...,0.03,0.26,6.5,0.2,0.0,2.7,0.02,0.01,0.25,0.04
4,119.7415,23.62,9.35,705.91,0.64,0.14,0.08,127.48,4.25,7.30,...,0.03,0.26,6.5,0.2,0.0,2.7,0.02,0.01,0.25,0.04


In [8]:
import sys
import xgboost
h2o.init()

train = h2o.H2OFrame(train_df)
test = h2o.H2OFrame(test_df)

# AutoML 実行（ここでは最大60秒間）
aml = H2OAutoML(max_runtime_secs=60, seed=1)
aml.train(y="OV", training_frame=train)

# 最も高性能なモデル（**leader model**）を取得し、テストデータで評価
leader = aml.leader
perf = leader.model_performance(test_data=test)
print(perf)
# leader（リーダーモデル）が何かを確認するには、type() や summary() を使います。
# leaderのタイプ（モデル種別）を表示
print(type(leader))

# モデルのサマリーを表示して詳細を確認（たとえばGBM, GLM, DeepLearning, などと表示される）
leader.summary()

# どんなモデルかざっくり確認（モデルIDや説明）
print(leader.algo)   # 'gbm', 'glm', 'deeplearning' など
print(leader.model_id)



Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "25.0.1" 2025-10-21; OpenJDK Runtime Environment Homebrew (build 25.0.1); OpenJDK 64-Bit Server VM Homebrew (build 25.0.1, mixed mode, sharing)
  Starting server from /Users/yutotakagi/Develop/ass-production-management2/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/3b/3hxgpzc57b70j4b1d7my6wfc0000gn/T/tmp080mb4sd
  JVM stdout: /var/folders/3b/3hxgpzc57b70j4b1d7my6wfc0000gn/T/tmp080mb4sd/h2o_yutotakagi_started_from_python.out
  JVM stderr: /var/folders/3b/3hxgpzc57b70j4b1d7my6wfc0000gn/T/tmp080mb4sd/h2o_yutotakagi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Tokyo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,1 month and 21 days
H2O_cluster_name:,H2O_from_python_yutotakagi_mdur0g
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.984 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
18:12:47.353: AutoML: XGBoost is not available; skipping it.
18:12:47.366: _train param, Dropping bad and constant columns: [X23]


18:12:47.749: _train param, Dropping bad and constant columns: [X23]

█
18:12:49.23: _train param, Dropping unused columns: [X23]
18:12:49.158: _train param, Dropping bad and constant columns: [X23]

██
18:12:50.96: _train param, Dropping bad and constant columns: [X23]
18:12:50.812: _train param, Dropping bad and constant columns: [X23]

█
18:12:51.467: _train param, Dropping bad and constant columns: [X23]

█
18:12:51.985: _train param, Dropping unused columns: [X23]
18:12:52.104: _train param, Dropping unused columns: [X23]
18:12:52.225: _train param, Dropping bad and constant columns: [X23]

███
18:12:53.361: _train param, Dropping bad and constan

In [14]:
print(leader.algo) 

gbm


In [10]:
# OV分布
train["OV"].summary()
test["OV"].summary()

/var/folders/3b/3hxgpzc57b70j4b1d7my6wfc0000gn/T/ipykernel_54894/1139172270.py:2: H2ODeprecationWarning: ``summary()`` is deprecated, please use ``show_summary()`` or ``get_summary()`` instead
  train["OV"].summary()


,OV
type,real
mins,52.56268
mean,156.03368903030295
maxs,418.4056
sigma,54.6797035997881
zeros,0
missing,0
0,111.2886
1,201.5617
2,111.941


,OV
type,real
mins,31.38486
mean,102.31979334000006
maxs,395.7734
sigma,67.24780558828951
zeros,0
missing,0
0,130.1202
1,115.6764
2,104.8819


In [11]:
perf = leader.model_performance(test_data=test)
print(perf)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 5812.25142039866
RMSE: 76.23812314320612
MAE: 67.92062194142275
RMSLE: 0.656052249504877
Mean Residual Deviance: 5812.25142039866


In [12]:
leader

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20260114_181247_model_9


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    59                 59                          17388                  5            5            5             10            26            18.8136

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 567.136830625653
RMSE: 23.814634799334065
MAE: 16.78796575306814
RMSLE: 0.1504938458799111
Mean Residual Deviance: 567.136830625653

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 730.8641670623624
RMSE: 27.03449957114728
MAE: 18.99601013678171
RMSLE: 0.17162744731686436
Mean Residual Deviance: 730.8641670623624

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
aic                     nan       0          nan           nan           nan           nan           nan
loglikelihood           nan       0          nan           nan           nan           nan           nan
mae                     18.9881   1.80612    17.5444       18.0391       17.5258       21.4128       20.4184
mean_residual_deviance  731.437   138.369    603.018       749.459       582.746       906.238       815.722
mse                     731.437   138.369    603.018       749.459       582.746       906.238       815.722
r2                      0.753278  0.0541597  0.806277      0.732878      0.810546      0.683947      0.732744
residual_deviance       731.437   138.369    603.018       749.459       582.746       906.238       815.722
rmse                    26.9475   2.56648    24.5564       27.3763       24.1401       30.1038       28.5608
rmsle                   0.170896  0.0170955  0.158016      0.164274      0.156448      0.197206      0.178536

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2026-01-14 18:12:59  3.865 sec   0                  54.656           45.3147         2987.28
    2026-01-14 18:12:59  3.873 sec   5                  39.9702          32.0025         1597.62
    2026-01-14 18:12:59  3.883 sec   10                 32.2552          24.4817         1040.4
    2026-01-14 18:12:59  3.892 sec   15                 28.2777          20.7758         799.63
    2026-01-14 18:12:59  3.900 sec   20                 26.584           19.0744         706.711
    2026-01-14 18:12:59  3.907 sec   25                 25.5051          18.1742         650.512
    2026-01-14 18:12:59  3.914 sec   30                 24.8843          17.6489         619.228
    2026-01-14 18:12:59  3.921 sec   35                 24.4929          17.3317         599.903
    2026-01-14 18:12:59  3.927 sec   40                 24.3054          17.1732         590.753
    2026-01-14 18:12:59  3.933 sec   45                 24.13            17.0915         582.258
    2026-01-14 18:12:59  3.941 sec   50                 24.0304          16.9993         577.461
    2026-01-14 18:12:59  3.947 sec   55                 23.9007          16.871          571.244
    2026-01-14 18:12:59  3.952 sec   59                 23.8146          16.788          567.137

Variable Importances: 
variable    relative_importance    scaled_importance       percentage
----------  ---------------------  ----------------------  ----------------------
X34         2233030.5              1.0                     0.1579665

In [13]:
# leader（リーダーモデル）が何かを確認するには、type() や summary() を使います。
# leaderのタイプ（モデル種別）を表示
print(type(leader))

# モデルのサマリーを表示して詳細を確認（たとえばGBM, GLM, DeepLearning, などと表示される）
leader.summary()

# どんなモデルかざっくり確認（モデルIDや説明）
print(leader.algo)   # 'gbm', 'glm', 'deeplearning' など
print(leader.model_id)


<class 'h2o.estimators.gbm.H2OGradientBoostingEstimator'>
gbm
GBM_grid_1_AutoML_1_20260114_181247_model_9
